In [ ]:
import os
import json
import numpy as np
import pandas as pd
#import transformers as tran
import tensorflow as tf
import tqdm
import random
import transformers

from tqdm import tqdm

# 데이터 구성
랜덤 연결 문장과 다음 연결 문장 모두 10000개의 폴더에 담겨있음.<br>
모든 설명에서 next prediction시에 참인 문장들은 'next'로 표현되며, 거짓인 문장들은 'random'으로 표현합니다.

In [ ]:
def make_json_list(n_path, r_path):
  '''
  n_path_list: next 문장들이 저장된 json folder 주소
  r_path_list: random 문장들이 저장된 json folder 주소
  '''
  n_path_list = os.listdir(n_path)
  r_path_list = os.listdir(r_path)
  return n_path_list, r_path_list

In [ ]:
def check_file_sen_len(file_path, path_list, dont_want_all_print=False):
  '''
  폴더에 저장된 파일의 문장의 갯수를 파일별로 출력합니다.
  모두 출력후, 모든 파일의 문장 갯수의 합과 평균을 출력합니다.
  dont_want_all_print: True일 경우, 모든 파일의 합과 평균만 출력합니다. 기본값은 False.
  '''
  sum_sen_cnt = 0
  num_file = len(path_list)
  for index in path_list:
    with open(file_path + '/' + index, 'r') as f:
      json_file = json.load(f)
    if dont_want_all_print == False:
      print(f"{index} 문장 갯수: {len(json_file['sentence'])}")
    sum_sen_cnt += len(json_file['sentence'])
  
  print(f"총 문장 갯수: {sum_sen_cnt}")
  print(f"평균 문장 갯수: {sum_sen_cnt/num_file}")

In [ ]:
def make_segment(sen_list, max_len, padding_value, n_or_r=True):
  '''
  segment와 label_cls를 추가합니다. n_or_r 값을 통해 cls값을 추가할 수 있습니다.
  sen_list 구성: (([sen1, label1], [sen2, label2]), ......)
  n_or_r: next sen list일 경우 True. random일 경우 False
  '''
  #######sep 확인 후 코드 수정이 필요########
  new_sen_list = []
  segments = []
  labels_cls = []
  labels_lm = []
  for data in sen_list:
    data1, data2 = data
    sen1, label1 = data1
    sen2, label2 = data2
    if len(sen1) != len(label1): #첫 코드 실수로 길이가 안맞음... sentence에서 [cls] 토큰 빼는 것을 까먹음
      sen1 = sen1[1:]
    if len(sen2) != len(label2):
      sen2 = sen2[1:]
    segment1 = np.zeros_like(sen1).tolist()
    segment2 = np.ones_like(sen2).tolist()
    sen = [[2] + sen1 + sen2]
    label_lm = [[-1] + label1 + label2]
    segment = [[0] + segment1 + segment2]
    # print("sen: " + str(len(sen[0])))
    # print("label: " + str(len(label_lm[0])))
    sen = tf.keras.preprocessing.sequence.pad_sequences(sen, maxlen=max_len, padding='post', value=padding_value)
    label_lm = tf.keras.preprocessing.sequence.pad_sequences(label_lm, maxlen=max_len, padding='post', value=-1) #ignore index: -1
    segment = tf.keras.preprocessing.sequence.pad_sequences(segment, maxlen=max_len, padding='post', value=padding_value)

    if n_or_r == True: #next일 경우
      new_sen_list.append(sen[0])
      segments.append(segment[0])
      labels_cls.append(1)
      labels_lm.append(label_lm[0])
    else: #random일 경우
      new_sen_list.append(sen[0])
      segments.append(segment[0])
      labels_cls.append(0)
      labels_lm.append(label_lm[0])
  return np.array(new_sen_list), np.array(segments), np.array(labels_cls), np.array(labels_lm)

In [ ]:
def make_dataset(n_file_index, r_file_index, n_path_list, r_path_list, n_path, r_path, max_len, padding_value):
  '''
  n_file_index: n_path_list 중 하나의 index를 나타냅니다.
  r_file_index: r_path_list 중 하나의 index를 나타냅니다.
  n_path_list: next json 파일의 경로 list파일입니다.
  r_path_list: random json 파일의 경로 list파일입니다.
  n_path: next 파일들이 저장되어 있는 folder의 경로입니다.
  n_path: random 파일들이 저장되어 있는 folder의 경로입니다.
  '''
  with open(n_path + '/' + n_path_list[n_file_index], 'r') as n_file:
    n_json_file = json.load(n_file)
  with open(r_path + '/' + r_path_list[n_file_index], 'r') as r_file:
    r_json_file = json.load(r_file)

  np.random.shuffle(r_json_file['sentence']) #step1 전처리에서 분야별로 파일에 축적이 되기 때문에, 다른 분야와도 섞일수 있도록 합니다.

  n_sen_list = n_json_file['sentence'] #-> 구성 ([sen, label], [sen, label])
  r_sen_list = [] #-> 구성 [sen, label] 위와 같이 변경 필요.
  for index, sen in enumerate(r_json_file['sentence']): 
    if index % 2 == 0:
      temp_sen = sen
    else: #2문장씩 묶기
      sen = (temp_sen, sen) #-> 구성 ([sen, label], [sen, label])으로 변경
      r_sen_list.append(sen)
  n_sen_list, n_segments, n_labels_cls, n_labels_lm = make_segment(n_sen_list, max_len, padding_value, True) #-> segment와 label_cls 추가. 구성 (sen, segment, label_cls, label_lm)으로 변경
  r_sen_list, r_segments, r_labels_cls, r_labels_lm = make_segment(r_sen_list, max_len, padding_value, False)

  sen_list = np.concatenate([n_sen_list, r_sen_list], axis=0)
  segments = np.concatenate([n_segments, r_segments], axis=0)
  labels_cls = np.concatenate([n_labels_cls, r_labels_cls], axis=0)
  labels_lm = np.concatenate([n_labels_lm, r_labels_lm], axis=0)
  return sen_list, segments, labels_cls, labels_lm

In [ ]:
#sen_list, segments, labels_cls, labels_lm = make_dataset(0, 0, n_path_list, r_path_list, n_path, r_path, 512, 0)